## alteration attack

In [53]:
# parameters
mu = 0
sigma = 20

#hiperparameters
k = 500
n = 300
p = 1.5*sigma
ratio_g_r = 3
gamma = ratio_g_r / (ratio_g_r + 1)

In [54]:
# 攻击
import pandas as pd
import numpy as np

# 加载数据
data = pd.read_csv('/home/zhengyihao/TabularMark/classfication_task/normal_distribution/watermarked_synthetic_dataset_labeled.csv')

# 确定要扰动数据的数量
alter_percentages = [0, 0.2, 0.4, 0.6, 0.8, 1.0]

for alter_percentage in alter_percentages:
    # 创建数据的深拷贝来扰动，这样就不会影响原始数据
    altered_data = data.copy()

    # 扰动数据
    num_alter = int(len(data) * alter_percentage)
    # print(num_alter)
    indices_to_alter = np.random.choice(data.index, size=num_alter, replace=False)

        # 对每个要扰动的数据进行操作
    for idx in indices_to_alter:
        alteration = np.random.uniform(-p, p)
        altered_data.loc[idx, 'dimension_0'] += alteration


    # 保存结果
    altered_data.to_csv(f'/home/zhengyihao/dataset/normal_distribution/tabularmark_alter{int(alter_percentage*100)}.data.csv', index=False)

## Detect

In [56]:
import math
import numpy as np
import pandas as pd

# 确定要扰动数据的数量
alter_percentages = [0, 0.2, 0.4, 0.6, 0.8, 1.0]

original_data = pd.read_csv("synthetic_dataset_labeled.csv")

# 加载索引列表
with open('tabularmark_index.txt', 'r') as f:
    indices = [int(line.strip()) for line in f.readlines()]

# 加载种子列表
with open('tabularmark_seed.txt', 'r') as f:
    seeds = [int(line.strip()) for line in f.readlines()]

for alter_percentage in alter_percentages:

    # 读取相应的数据集
    data = pd.read_csv(f'/home/zhengyihao/dataset/normal_distribution/tabularmark_alter{int(alter_percentage*100)}.data.csv')

    green_cell = 0

    for idx, seed in zip(indices, seeds):
        np.random.seed(seed)
        intervals = np.linspace(-p, p, k+1)
        unit_domains = [(intervals[i], intervals[i+1]) for i in range(k)]
        np.random.shuffle(unit_domains)

        # 分割domains为green domains和red domains
        green_domains = unit_domains[:int(len(unit_domains)/ (ratio_g_r + 1) * ratio_g_r)]
        red_domains = unit_domains[int(len(unit_domains)/ (ratio_g_r + 1) * ratio_g_r):]


        deviation = data.loc[idx, 'dimension_0'] - original_data.loc[idx, 'dimension_0']

        # 初始设定 deviation 不在 green_domains 中
        in_green_domains = False

        # 遍历每一个 green_domains 元素
        for domain in green_domains:
            # 如果 deviation 在当前的 domain（区间）内
            if domain[0] <= deviation < domain[1]:
                # 设置 in_green_domains 为 True，然后跳出循环
                in_green_domains = True
                break

        if in_green_domains :
            green_cell += 1

    print(f"For alteration percentage {alter_percentage*100}%, green cells: {green_cell}")

    # calculate z-score

    z_score = (green_cell - n*gamma) / math.sqrt(n*(gamma)*(1-gamma))

    print("The z-score is ",z_score)
    print("The percentage of green celsl is ", green_cell/ n)    


For alteration percentage 0%, green cells: 213
The z-score is  -1.6
The percentage of green celsl is  0.71
For alteration percentage 20.0%, green cells: 213
The z-score is  -1.6
The percentage of green celsl is  0.71
For alteration percentage 40.0%, green cells: 213
The z-score is  -1.6
The percentage of green celsl is  0.71
For alteration percentage 60.0%, green cells: 223
The z-score is  -0.26666666666666666
The percentage of green celsl is  0.7433333333333333
For alteration percentage 80.0%, green cells: 213
The z-score is  -1.6
The percentage of green celsl is  0.71
For alteration percentage 100.0%, green cells: 219
The z-score is  -0.8
The percentage of green celsl is  0.73
